## Análise Exploratória de Dados

Uma empresa solicitou a uma consultoria um diagnóstico de seu Serviço de Suporte a Clientes. Com base no diagnóstico a partir da análise de dados, a empresa pretende entender quais são as oportunidades de melhorias dos produtos e serviços ofertados.

### Objetivo

O objetivo deste notebook é aplicar as principais funções em Python para conduzir a análise de dados, dentre eles:

- Medidas resumo
- Tabela de frequência
- Gráficos

Considerando a base suporte_tecnico.xlsx, os dados serão extraídos para auxiliar o diagnóstico.

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# Lendo a base de dados
df = pd.read_excel('C:\_Cursos\Python_Analise_Dados/suporte_tecnico.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3025 entries, 0 to 3024
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   ticket_id                      3025 non-null   int64         
 1   inicio_atendimento             3025 non-null   datetime64[ns]
 2   fim_atendimento                3025 non-null   datetime64[ns]
 3   turno                          3025 non-null   object        
 4   atendente_nome                 3025 non-null   object        
 5   resolvido_na_primeira_chamada  3025 non-null   object        
 6   conversao_atendimento          3025 non-null   object        
 7   nota_csat                      3006 non-null   float64       
 8   produto                        3025 non-null   object        
 9   cliente                        3025 non-null   object        
 10  ticket_reaberto                3025 non-null   object        
 11  ticket_original  

In [3]:
df.head()

,ticket_id,inicio_atendimento,fim_atendimento,turno,atendente_nome,resolvido_na_primeira_chamada,conversao_atendimento,nota_csat,produto,cliente,ticket_reaberto,ticket_original
0,1048,2023-12-01 06:00:00,2023-12-01 06:15:00,Manhã,Carla Souza,Sim,Sim,7.0,Serviço C,Empresa X,Não,NaN
1,436,2023-12-01 06:12:00,2023-12-01 06:18:00,Manhã,Ana Paula,Sim,Sim,4.0,Serviço C,Empresa Z,Não,NaN
2,1725,2023-12-01 06:18:00,2023-12-01 06:52:00,Manhã,Maria Fernandes,Não,Não,3.0,Produto B,Empresa Y,Sim,NaN
3,1186,2023-12-01 06:19:00,2023-12-01 06:56:00,Manhã,Carla Souza,Sim,Sim,4.0,Serviço C,Empresa X,Não,NaN
4,2683,2023-12-01 06:27:00,2023-12-01 06:50:00,Manhã,Ricardo Andrade,Não,Não,2.0,Produto A,Empresa Z,Não,NaN


In [4]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
3020    False
3021    False
3022    False
3023    False
3024    False
Length: 3025, dtype: bool

##### a) TMA - 25%

Com base no Tempo Médio de Atendimento (TMA), qual seria o valor de tempo de atendimento que separaria os 25% dos tickets abertos?

In [5]:
# Criando uma coluna para o cálculo do TMA
df['TMA'] = df['fim_atendimento']-df['inicio_atendimento']

In [6]:
df['TMA'].describe()

count                         3025
mean     0 days 00:33:18.069421487
std      0 days 00:17:15.480476207
min                0 days 00:02:00
25%                0 days 00:18:00
50%                0 days 00:36:00
75%                0 days 00:48:00
max                0 days 01:00:00
Name: TMA, dtype: object

##### Análise: 

O tempo médio de atendimento (TMA) para o primeiro quartil (25% dos tickets abertos) é de 18 minutos. Ou seja, em 25% dos casos, o atendimento é finalizado em um tempo menor ou igual a 18 minutos.

##### b) Tickets por turno de trabalho

Há alguma concentração de tickets em algum turno de trabalho?

In [7]:
# Calculando a frequencia de tickets por turno de trabalho
tabela_freq_turno = df['turno'].value_counts().reset_index() 
tabela_freq_turno.columns = ['Turno', 'Frequência Absoluta']
tabela_freq_turno['Frequência Relativa']=tabela_freq_turno['Frequência Absoluta']/tabela_freq_turno['Frequência Absoluta'].sum()
tabela_freq_turno['Frequência Acumulada']=tabela_freq_turno['Frequência Relativa'].cumsum()
tabela_freq_turno

,Turno,Frequência Absoluta,Frequência Relativa,Frequência Acumulada
0,Noite,1289,0.426116,0.426116
1,Manhã,1022,0.337851,0.763967
2,Tarde,714,0.236033,1.000000


##### Análise:

Observa-se que o turno noturno concentra o maior percentual de atendimentos, ou seja, 42% dos atendimentos (1.289 tickets).

##### c) Criação de gráfico de barras

In [8]:
# Cálculo da média das notas por atendendente
medias = df.groupby('atendente_nome')['nota_csat'].mean().reset_index()
print(medias)

    atendente_nome  nota_csat
0        Ana Paula   5.435039
1      Carla Souza   5.403614
2       João Silva   5.563241
3    Lucas Martins   5.699422
4  Maria Fernandes   5.297830
5  Ricardo Andrade   5.510684


In [11]:
fig = px.bar(medias, x="atendente_nome", y="nota_csat", title="Média das Avaliações por Atendente", 
             color_discrete_sequence=["#419873"])
fig.update_layout(
    plot_bgcolor="rgba(0,0,0,0)",
    width=800,
    height=400,
    xaxis_title="Atendente",
    yaxis_title="Média Avaliações"
)

##### d) Taxa de Conversão de Atendimento

Qual foi a taxa de conversão de atendimento?

In [10]:
(df['conversao_atendimento'].value_counts()*100/df['conversao_atendimento'].count()).apply('{:.1f}%'.format)

Não    51.2%
Sim    48.8%
Name: conversao_atendimento, dtype: object

##### Análise:

Para 51,2% dos casos, não houve conversão de atendimento, ou seja, o cliente não considera que aquele atendimento resolveu o problema.